In [1]:
import json
import numpy as np
import pandas as pd
import os
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
from tqdm.notebook import tqdm
import torch.nn as nn
import torchvision.models as models
from torchvision.models import ResNet50_Weights, ResNet101_Weights
from pathlib import Path


In [2]:
!pip install ultralytics --quiet
!pip install dill

In [3]:
from ultralytics import YOLO

## Downlaod Test Data  and Unzip

In [4]:
!gdown --id '1KdNjCIot9SYh9JxRHuCjrPdmgUDazZ9U'
!unzip test.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1KdNjCIot9SYh9JxRHuCjrPdmgUDazZ9U
From (redirected): https://drive.google.com/uc?id=1KdNjCIot9SYh9JxRHuCjrPdmgUDazZ9U&confirm=t&uuid=7ffcbc3d-5464-4aac-a7c6-ccf82f8b7cf0
To: /content/test.zip
100% 2.92G/2.92G [00:23<00:00, 125MB/s]
Archive:  test.zip
replace test/3910_[40.70800769388502, -89.38938456054692]_ 2023-10-18 16-18-59_22.8697_LD.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test/3910_[40.70800769388502, -89.38938456054692]_ 2023-10-18 16-18-59_22.8697_LD.jpg  
  inflating: test/6780_(40.7029723333333, -89.4051231666667)_ 2023-10-18 17-02-15_4.22725_D.jpg  
  inflating: test/1101_(40.7033468333333, -89.40616)_ 2023-10-18 15-48-47_16.61855_D.jpg  
  inflating: test/20513_(40.7089811666667, -8

## Change to the path to your dataset

In [5]:
PATH_TO_TEST_DATASET = 'test'


## Create Test Dataset for Inference


In [15]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

class PCITestDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_names = [img for img in os.listdir(img_dir) if
                          img.endswith(('jpg', 'png', 'jpeg'))]

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_name = self.img_names[idx]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except IOError as e:
            print(f"Error loading image {img_path}: {e}. Skipping.")
            return None, None

        if self.transform:
            image = self.transform(image)

        return image, img_name

## Create Test Data Transform and DataLoader

In [16]:
# Defining the transform for the test set
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


# Create the test dataset
test_dataset = PCITestDataset(img_dir=PATH_TO_TEST_DATASET, transform=test_transform)

# Create the DataLoader for the test set
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

## Download ResNet and the YOLO Model checkpoints

In [8]:
!gdown --id '1Jk10bgNx9w4FoJJDi-F2nS6kUhRR_Iv3' #ResNet50
!gdown --id '1m-DWqJTdERL_G9M1nRbbVxav8cokTb2a' # ResNet101
!gdown --id '1q9hR1XHXMjwb68VOOM83ZZBYNnOj2MvR' #yolov8-cls

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1Jk10bgNx9w4FoJJDi-F2nS6kUhRR_Iv3
From (redirected): https://drive.google.com/uc?id=1Jk10bgNx9w4FoJJDi-F2nS6kUhRR_Iv3&confirm=t&uuid=8430dded-74a2-4bb6-a7e2-af129d32f436
To: /content/best_model_resnet50_full_dataset_random_split_v3.pth
100% 283M/283M [00:02<00:00, 133MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1m-DWqJTdERL_G9M1nRbbVxav8cokTb2a
From (redirected): https://drive.google.com/uc?id=1m-DWqJTdERL_G9M1nRbbVxav8cokTb2a&confirm=t&uuid=24b24bec-0028-4cd3-b4d6-c7e121d1d8f

In [17]:
!ls

best_model_resnet50_full_dataset_random_split_v3.pth  test
best_resnet101_full_dataset_random_split_v3.pth       test.zip
sample_data					      yolovl8_batch64_lr_0.0001_augment.pt


## Define model paths

> All models will be saved in the current directory

In [18]:
path_to_resnet_50 = 'best_model_resnet50_full_dataset_random_split_v3.pth'
path_to_resnet101 = 'best_resnet101_full_dataset_random_split_v3.pth'
path_to_yolov8 = 'yolovl8_batch64_lr_0.0001_augment.pt'

Create a function to load a resnet checkpoint

In [19]:
def return_resnet_model_checkpoint(model_type, model_path):
    if model_type == 'resnet50':
        model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
    elif model_type == 'resnet101':
        model = models.resnet101(weights=ResNet101_Weights.DEFAULT)

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 1)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_path = torch.load(model_path, map_location=device)

    model.load_state_dict(model_path['model_state_dict'])
    model = model.to(device)

    return model

## Load models for inference

In [20]:
resnet50_model =  return_resnet_model_checkpoint('resnet50', path_to_resnet_50)
resnet101_model =  return_resnet_model_checkpoint('resnet101', path_to_resnet101)
yolov8_model = YOLO(path_to_yolov8)

Function to return dataframe for resnet models

In [21]:
def return_resnet_model_df(model, test_loader, device):
    '''
    model: - model checkpoint for inference
    test_laoder - dataloader for test data

    Returns dataframe with two columns:
    'image_name'
    'PCI'
    '''
    model.eval()
    predictions = []

    with torch.no_grad():
        for data, file_names in tqdm(test_loader):
            data = data.to(device)
            outputs = model(data).squeeze()

            outputs = outputs.cpu().numpy()

            outputs = np.round(outputs)
            outputs = np.clip(outputs, 0, 100)

            for file_name, output in zip(file_names, outputs):
                predictions.append({'image_name': file_name, 'PCI': output})

    return pd.DataFrame(predictions)

Function to return dataframe for YOLOv8-cls model

In [22]:
def return_YOLO_model_df(test_dir, yolo_model):
    rows = []
    test_path = Path(test_dir)
    for tst_img in tqdm(test_path.glob('**/*.jpg'), description='Running inference for YOLOv8-cls-model'):
        preds = yolo_model(tst_img)
        cls_dict = preds[0].names
        probs = preds[0].probs.data.cpu().numpy()
        rows.append({'image_name':os.path.basename(tst_img),
                    'PCI':cls_dict[np.argmax(probs)]})
    df_test = pd.DataFrame(rows)
    return df_test

### Generate dataframe with the columns:

`image_name` <br>
`PCI`

## Get dataframes for ResNet models

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Get the dataframe for ResNet50
resnet50_df = return_resnet_model_df(resnet50_model, test_loader, device)
# Get the dataframe for ResNet101
resnet101_df =  return_resnet_model_df(resnet101_model, test_loader, device)

  0%|          | 0/52 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Get dataframe for YOLOv8l-cls model

In [ ]:
YOLO_model_df = return_YOLO_model_df(PATH_TO_TEST_DATASET, yolov8_model)

0it [00:00, ?it/s]

Streaming output truncated to the last 5000 lines.
Speed: 25.4ms preprocess, 11.7ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/test/22017_(40.7113573333333, -89.417521)_ 2023-10-20 19-25-33_37.6031_D.jpg: 224x224 100 0.72, 64 0.24, 91 0.01, 49 0.01, 97 0.00, 11.3ms
Speed: 24.6ms preprocess, 11.3ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/test/10587_[40.723674078300895, -89.43942715832083]_ 2023-10-18 20-30-09_38.87035_LD.jpg: 224x224 62 0.39, 48 0.37, 64 0.08, 41 0.05, 49 0.02, 11.0ms
Speed: 25.5ms preprocess, 11.0ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/test/41826_[40.704726769705324, -89.46030748529772]_ 2023-10-20 16-42-12_25.86115_LD.jpg: 224x224 8 0.22, 41 0.10, 4 0.10, 25 0.09, 19 0.09, 11.0ms
Speed: 24.2ms preprocess, 11.0ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/test/34979_(40.7148673333333, -89.430791

# Ensemble Model Results

In [25]:
PERCENTILE = 0.001

In [ ]:
# Sort data
resnet50_df = resnet50_df.sort_values(by='image_name', ignore_index=True)
resnet101_df = resnet101_df.sort_values(by='image_name', ignore_index=True)
YOLO_model_df = YOLO_model_df.sort_values(by='image_name', ignore_index=True)

# Create a new dataframe for our ensemble
results = pd.DataFrame()
results['image_name'] = resnet50_df['image_name']

results['resnet50'] = resnet50_df['PCI']
results['resnet101'] = resnet101_df['PCI']
results['yolov8l_cls'] = YOLO_model_df['PCI']

# cols = ['resnet50', 'resnet101', 'yolov8l_cls']

results['PCI'] = results.iloc[:, 1:].apply(lambda row: np.percentile(row, PERCENTILE), axis=1)

### Generate JSON file for Submission

In [ ]:
def gen_submit(df, json_file_name):
    out_json = []
    for idx, results in df.iterrows():
        out_json.append({results['image_name']: results['PCI']})
    with open(json_file_name, 'w') as f:
        json.dump(out_json, f)

## Generate final JSON file for ensemble results

The final json file for submission called `submission.json` will be saved in your current directory

In [ ]:
gen_submit(results, 'submission.json')